In [85]:
import numpy as np
import pandas as pd
import csv
import mysql.connector
from datetime import date, datetime
import random
import time

In [27]:
class Lotofacil:
    def __init__(self):
        self.id = 0
        self.data_sorteio = None
        self.ganhadores_15_numeros = 0
        self.valor_rateio_15_numeros = 0
        self.valor_rateio_14_numeros = 0
        self.valor_rateio_13_numeros = 0
        self.valor_rateio_12_numeros = 0
        self.valor_rateio_11_numeros = 0
        self.par = 0
        self.impar = 0
        self.numeros = ()
       
    def setParImpar(self):
        self.par = sum([1 if x % 2 == 0 else 0 for x in self.numeros])
        self.impar = len(self.numeros) - self.par

In [28]:
class Jogo:
    def __init__(self):
        self.id = None
        self.data_cadastro = None
        self.quantidade = 0
        self.par = 0
        self.impar = 0
        numeros = ()
        
    def sorteio(self, qtdNumeros = 15):
        self.numeros = sorted(tuple(np.random.choice(np.arange(1, 26), qtdNumeros, replace= False)))
        self.setQuantidade()
        self.setParImpar()
        
    def setQuantidade(self):
        self.quantidade = len(self.numeros)
        
    def setParImpar(self):
        self.par = sum([1 if x % 2 == 0 else 0 for x in self.numeros])
        self.impar = len(self.numeros) - self.par

In [29]:
def get_concursos_lotofacil_csv(arquivo):
    concursos = []
    
    with open(arquivo) as file:
        lotofacil = csv.reader(file, delimiter=';')
        data = list(lotofacil)
    
    for x in data[1:]:
        concurso = Lotofacil()
        concurso.id = int(x[0])
        concurso.data_sorteio = x[1]
        concurso.numeros = sorted(list(map(int, x[2:17])))
        concurso.ganhadores_15_numeros = int(x[17])
        concurso.valor_rateio_15_numeros = float(x[18].replace('.', '').replace(',', '.'))
        concurso.valor_rateio_14_numeros = float(x[19].replace('.', '').replace(',', '.'))
        concurso.valor_rateio_13_numeros = float(x[20].replace('.', '').replace(',', '.'))
        concurso.valor_rateio_12_numeros = float(x[21].replace('.', '').replace(',', '.'))
        concurso.valor_rateio_11_numeros = float(x[22].replace('.', '').replace(',', '.'))
        concurso.setParImpar()
        concursos.append(concurso)
    
    return concursos

In [57]:
def gerar_jogos(qtd_apostas = 1000, numeros_por_jogo = 15, menor_numero = (), maior_numero = ()):
    count = 0
    jogos = []
#     limite_inferior = 3
#     limite_superior = 23
    qtd_tentativas = 0
    
    while (count < qtd_apostas and qtd_tentativas < 1000):
        add = False
        jogo = Jogo()
        jogo.sorteio(numeros_por_jogo)
        
#         for menor in menor_numero:
#             for maior in maior_numero:
        if (jogo.numeros[0] not in menor_numero or jogo.numeros[-1] not in maior_numero):
            qtd_tentativas += 1
            continue

#         if (jogo.numeros[0] > limite_inferior or jogo.numeros[-1] < limite_superior):
#             qtd_tentativas += 1
#             continue

        for i in jogos:
            if (len(i.numeros) == len(jogo.numeros)):
                if (len(set(i.numeros).intersection(jogo.numeros)) == len(jogo.numeros)):
                    qtd_tentativas += 1
                    break
                
#         if (add):
        jogos.append(jogo)
        count += 1
        qtd_tentativas = 0
            
#     print('apostas geradas: %s - tentativas: %s' % (count, qtd_tentativas))
            
    return jogos

In [31]:
def filtrar_concursos_por_numero(concursos, inicial, final):
    concursos_filtrados = []
    
    for c in concursos:
        if (c.id >= inicial and c.id <= final):
            concursos_filtrados.append(c)
    
    return concursos_filtrados

In [32]:
def filtrar_ultimos_concursos(concursos, ultimos):
    concursos_filtrados = []
    
    if (ultimos is not None):
        concursos_filtrados = concursos[len(concursos)-ultimos:]
        
    return concursos_filtrados

In [33]:
def filtrar_concurso_por_numeros(concursos, numeros):
    concursos_filtrados = []
    
    for c in concursos:
        if (len(set(c.numeros).intersection(numeros)) == len(numeros)):
            concursos_filtrados.append(c)
#         if len(set(apostas[i].numeros).intersection(notin[j])) == len(notin[j]):
    
    return concursos_filtrados

In [34]:
def imprimir_concursos(concursos):
    print("Total concursos: %s \n" % (len(concursos)))
    for c in concursos:
        print("Concurso: ", c.id)
        print("Data Sorteio: ", c.data_sorteio)
        print("Ganhadores 15 pontos: %s " % (c.ganhadores_15_numeros))
        print("Impares: %s - Pares: %s" % (c.impar, c.par))
        print(c.numeros)
        print("\n")

In [35]:
def imprimir_aposta(apostas):
    print("Total apostas: %s \n" % (len(apostas)))
    for a in apostas:
#         print("ID: ", a.id)
#         print("Data: ", a.data_sorteio)
        print("Impares: %s - Pares: %s" % (a.impar, a.par))
        print(a.numeros)
        print("\n")

In [36]:
def imprimir_jogo(jogos):
    print("Total jogos: %s \n" % (len(jogos)))
    for a in jogos:
        print("ID: ", a.id)
        print("Data Cadastro: ", a.data_cadastro)
        print("Numeros: %s - Impares: %s - Pares: %s" % (len(a.numeros), a.impar, a.par))
        print(a.numeros)
        print("\n")

In [37]:
def filtrar_jogo_inicio_fim(jogos, posicao, numeros):
    apostas_filtradas = []

    for a in jogos:
        for n in numeros:
            if (a.numeros[posicao] == n):
                apostas_filtradas.append(a)
                continue
        
    return apostas_filtradas

In [38]:
def filtrar_jogo_impar_par(jogos, impar):
    apostas_filtradas = []
    
    for a in jogos:
        for i in impar:
            if (a.impar == i):
                apostas_filtradas.append(a)
        
    return apostas_filtradas

In [39]:
def filtrar_jogo_por_numeros(jogos, numeros, match):
    apostas_filtradas = []
    
    for j in jogos:
        if (len(set(j.numeros).intersection(numeros)) >= match):
            apostas_filtradas.append(j)
        
    return apostas_filtradas

In [40]:
def filtrar_jogo_sequencia(jogos, numeros):
    apostas_filtradas = []
    
    for j in jogos:
        if (len(set(j.numeros).intersection(numeros)) < len(numeros)):
            apostas_filtradas.append(j)
        
    return apostas_filtradas

In [41]:
def filtrar_jogo_notin(jogos, numeros):
    apostas_filtradas = []
    
    for j in jogos:
        if (len(set(j.numeros).intersection(numeros)) == 0):
            apostas_filtradas.append(j)
        
    return apostas_filtradas

In [42]:
def selecionar_jogos_aposta(jogos, quantidade):
    jogos_sample = random.sample(jogos, quantidade)
    menor_1 = 0
    menor_2 = 0
    menor_3 = 0
    impar_7 = 0
    impar_8 = 0
    impar_9 = 0

    for a in jogos_sample:
#         print(a.numeros, a.impar, a.par, a.numeros[0], a.numeros[-1])
        if a.numeros[0] == 1:
            menor_1 += 1
        elif a.numeros[0] == 2:
            menor_2 += 1
        elif a.numeros[0] == 3:
            menor_3 += 1

        if a.impar == 7:
            impar_7 += 1
        elif a.impar == 8:
            impar_8 += 1
        elif a.impar == 9:
            impar_9 += 1

    print('Percentual jogos com 1: ', (menor_1/len(jogos_sample))*100)
    print('Percentual jogos com 2: ', (menor_2/len(jogos_sample))*100)
    print('Percentual jogos com 3: ', (menor_3/len(jogos_sample))*100)
    print('Percentual jogos impar 7: ', (impar_7/len(jogos_sample))*100)
    print('Percentual jogos impar 8: ', (impar_8/len(jogos_sample))*100)
    print('Percentual jogos impar 9: ', (impar_9/len(jogos_sample))*100)

    return jogos_sample

In [43]:
def conexao_mysql():
    return mysql.connector.connect(user='root', database='lotofacil', host='192.168.56.100', password='Zaq1@wsX')

In [44]:
def get_numeros_jogo(jogo_id):
    numeros = []
    
    try:
        cnx = conexao_mysql()
        cursor = cnx.cursor()

        query = ("SELECT numero FROM numero_jogo "
                 "WHERE jogo_id = %(jogo_id)s "
                 "ORDER BY numero")
        
        param_query = {
            'jogo_id' : jogo_id,
        }
 
        cursor.execute(query, param_query)

        rows = cursor.fetchall()
        
        for row in rows:
            numeros.append(row[0])

    except Exception as ex:
        print(ex)
 
    except ConnectionError as e:
        print(e)
 
    finally:
        cursor.close()
        cnx.close()

    return tuple(numeros)

In [87]:
def get_all_jogos():
    jogos = []
    cursor = None
    cnx = None
    
    try:
        cnx = conexao_mysql()
        cursor = cnx.cursor()

        query = ("SELECT jogo_id, data_cadastro "
                 "FROM jogo "
#                  "WHERE jogo_id < 10001 "
                 "ORDER BY jogo_id")
        
        cursor.execute(query)
        
        rows = cursor.fetchall()
        
        for row in rows:
            jogo = Jogo()
            jogo.id = row[0]
            jogo.data_cadastro = row[1]
            jogo.numeros = get_numeros_jogo(row[0])
            jogo.setParImpar()
            jogo.setQuantidade()
            jogos.append(jogo)
    #         print(jogo_id, data_cadastro)

    except Exception as ex:
        print(ex)
        
    except ConnectionError as e:
        print(e)
        
    finally:
        cursor.close()
        cnx.close()
    
    return jogos

In [98]:
def insert_jogo(jogo):
    try:
        cnx = conexao_mysql()
        cursor = cnx.cursor()
       
        add_jogo = ("INSERT INTO jogo "
                     "(data_cadastro) "
                     "VALUES (%(data_cadastro)s)")
        add_numero_jogo = ("INSERT INTO numero_jogo "
                            "(jogo_id, numero) "
                            "VALUES (%(jogo_id)s, %(numero)s)")

        data_jogo = {
            'data_cadastro' : datetime.now().date(),
        }

        cursor.execute(add_jogo, data_jogo)
        jogo_id = cursor.lastrowid
        
#         numeros = [(jogo_id, numero) for numero in j.numeros]

        for numero in jogo.numeros:
            data_numero_jogo = {
              'jogo_id': jogo_id,
              'numero': int(numero),
            }
            
            cursor.execute(add_numero_jogo, data_numero_jogo)

#         data_numero_jogo = {
#           'jogo_id': jogo_id,
#           'numero': jogo.numeros,
#         }

#         cursor.execute(add_numero_jogo, data_numero_jogo)
        
        cnx.commit()
        
    except Exception as ex:
        cnx.rollback()
        return ex
        
    finally:
        cursor.close()
        cnx.close()

In [90]:
def salvar_jogos(jogos):
    total_salvos = 0
    
    try:
        jogos_db = get_all_jogos()
        
        for jogo in jogos:
            add = True
            
            for jogo_db in jogos_db:
                if (len(jogo.numeros) == len(jogo_db.numeros)):
                    if (len(set(jogo.numeros).intersection(jogo_db.numeros)) == len(jogo.numeros)):
                        add = False
                        break

            if (add):
                insert_jogo(jogo)
                total_salvos += 1
                
    except Exception as ex:
        print(ex)
        
    finally:
        print("%s jogos gravados." % (total_salvos))
        print("%s jogos repetidos." % (len(jogos) - total_salvos))

In [17]:
# variaveis globais
# min_numeros_aposta = 15
# max_numeros_aposta = 18

In [54]:
# while True:
#     try:
#         numerosAposta = int(input('Quantidade de números por aposta: '))
#         if numerosAposta < 15 or numerosAposta > 18:
#             raise ValueError('Digite um valor entre 15 e 18!')
#     except ValueError as e:
#         print(e)
#         continue
#     except:
#         print('Erro desconhecido!')
#         continue
#     else:
#         break

# Gerar jogos

In [60]:
jogos_nao_salvos = gerar_jogos(100, menor_numero=(1,2), maior_numero=(24,25))
len(jogos_nao_salvos)

100

In [62]:
# imprimir_jogo(jogos_nao_salvos)

In [91]:
start = time.time()
salvar_jogos(jogos_nao_salvos)
end = time.time()

print((end - start)/60)

94 jogos gravados.
6 jogos repetidos.
75.17318996191025


In [88]:
start = time.time()
jogos_salvos = get_all_jogos()
end = time.time()

print((end - start)/60)
print(len(jogos_salvos))

74.07931682268779
100000


In [99]:
# insert_jogo(jogos_nao_salvos[0])

In [11]:
i = 0

while i < len(apostas):
    j = 0
    while j < len(apostas):
        if i != j:
            if len(set(apostas[i].numeros).intersection(apostas[j].numeros)) == 15:
                del apostas[j]
        j += 1
#     print(len(set(a).intersection(apostas[i])))
    i += 1

In [13]:
i = 0
notin = [[1,2,3,4,5,6,7,8], [18,19,20,21,22,23,24,25]]

while i < len(apostas):
    j = 0
    a = 0
    while j < len(notin):
        if len(set(apostas[i].numeros).intersection(notin[j])) == len(notin[j]):
            a += 1
        j += 1
    
    if (a > 0):
        del apostas[i]
        
    i += 1

In [14]:
i = 0
final = [23,24,25]

while i < len(apostas):
    if len(set(apostas[i].numeros).intersection(final)) < 2:
        del apostas[i]
        
    i += 1

In [15]:
i = 0
inicio = [1,2,3]

while i < len(apostas):
    if len(set(apostas[i].numeros).intersection(inicio)) < 2:
        del apostas[i]
        
    i += 1

In [16]:
i = 0

while i < len(apostas):
    if (apostas[i].impar <= 6 or apostas[i].impar >= 10):
        del apostas[i]
    i += 1

In [ ]:
# !pip install urllib urllib2

In [ ]:
# import urllib

In [ ]:
# page = urllib.request.urlopen("file:///Users/raphael/Documents/GitHub/python_scripts/D_LOTFAC.HTM", 'r')

In [ ]:
# from bs4 import BeautifulSoup

In [ ]:
# soup = BeautifulSoup(page.read(), 'html.parser')

In [ ]:
# soup = BeautifulSoup(page.read(), 'lxml')

In [ ]:
# table = soup.find_all('table')[0]

In [ ]:
# for row in table.find_all('tr'):
#     columns = row.find_all(['td', 'th'])
#     for col in columns:
#         print(col.text())

In [ ]:
# i = 1
# for row in soup.findAll('table')[0].findAll('tr'):
#     concurso = row.findAll('td')
#     print(concurso)
    
#     i += 1

In [110]:
concursos = get_concursos_lotofacil_csv('lotofacil_data/lotofacil.csv')

In [115]:
for x in concursos[:10:-1]:
    print(x.id, x.data_sorteio, x.ganhadores_15_numeros, x.valor_rateio_15_numeros , x.valor_rateio_14_numeros, x.valor_rateio_13_numeros, x.valor_rateio_12_numeros, x.valor_rateio_11_numeros, x.impar, x.par)
    print(x.numeros)

1653 23/04/2018 0 0.0 1336.9 20.0 8.0 4.0 7 8
[2, 4, 5, 6, 7, 12, 14, 15, 17, 20, 21, 22, 23, 24, 25]
1652 20/04/2018 0 0.0 2139.67 20.0 8.0 4.0 7 8
[2, 3, 4, 5, 6, 7, 10, 11, 14, 16, 17, 18, 21, 24, 25]
1651 18/04/2018 4 1386784.84 2757.56 20.0 8.0 4.0 5 10
[1, 2, 3, 4, 6, 7, 8, 10, 16, 17, 18, 19, 20, 22, 24]
1650 16/04/2018 0 0.0 1583.01 20.0 8.0 4.0 7 8
[3, 4, 5, 8, 10, 13, 14, 15, 16, 17, 18, 19, 20, 22, 25]
1649 13/04/2018 2 1040196.59 1696.58 20.0 8.0 4.0 6 9
[4, 5, 8, 9, 12, 13, 14, 15, 16, 17, 18, 20, 22, 23, 24]
1648 11/04/2018 1 2284989.88 1905.35 20.0 8.0 4.0 9 6
[4, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21, 22, 23, 24, 25]
1647 09/04/2018 5 290384.06 689.2 20.0 8.0 4.0 6 9
[1, 2, 4, 7, 8, 9, 10, 12, 14, 15, 19, 20, 22, 24, 25]
1646 06/04/2018 6 292923.84 805.57 20.0 8.0 4.0 9 6
[1, 3, 4, 5, 6, 7, 9, 12, 13, 15, 17, 18, 20, 22, 25]
1645 04/04/2018 3 614454.25 1540.43 20.0 8.0 4.0 7 8
[2, 3, 4, 5, 8, 9, 10, 14, 15, 16, 18, 20, 21, 23, 25]
1644 02/04/2018 2 692887.47 1340.85 20.

1522 09/06/2017 2 927234.52 1649.15 20.0 8.0 4.0 8 7
[1, 8, 9, 10, 11, 13, 14, 15, 16, 18, 19, 20, 21, 23, 24]
1521 07/06/2017 4 359073.26 1191.2 20.0 8.0 4.0 8 7
[1, 2, 3, 4, 5, 7, 8, 12, 14, 15, 17, 19, 20, 23, 24]
1520 05/06/2017 6 283547.62 1504.66 20.0 8.0 4.0 6 9
[1, 2, 4, 8, 9, 10, 12, 13, 14, 16, 17, 18, 19, 24, 25]
1519 02/06/2017 4 529678.45 1969.52 20.0 8.0 4.0 6 9
[1, 4, 6, 8, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25]
1518 31/05/2017 4 440990.32 1424.56 20.0 8.0 4.0 6 9
[2, 3, 4, 5, 7, 8, 11, 12, 14, 16, 17, 20, 22, 24, 25]
1517 29/05/2017 4 276609.09 701.79 20.0 8.0 4.0 7 8
[2, 3, 5, 8, 10, 11, 12, 14, 15, 18, 20, 21, 23, 24, 25]
1516 26/05/2017 3 676080.02 2080.25 20.0 8.0 4.0 5 10
[1, 2, 6, 7, 10, 11, 12, 14, 16, 18, 19, 20, 22, 23, 24]
1515 24/05/2017 5 336430.51 1559.0 20.0 8.0 4.0 9 6
[1, 2, 3, 5, 8, 9, 11, 12, 13, 16, 17, 18, 19, 20, 23]
1514 22/05/2017 2 858676.0 1793.06 20.0 8.0 4.0 9 6
[3, 4, 5, 7, 8, 11, 12, 13, 15, 16, 17, 19, 20, 21, 24]
1513 19/05/2017 1 193

1359 09/05/2016 11 306894.18 612.98 20.0 8.0 4.0 10 5
[3, 5, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 20, 21, 23]
1358 06/05/2016 0 0.0 1872.29 20.0 8.0 4.0 8 7
[1, 2, 3, 4, 6, 8, 9, 13, 15, 17, 18, 19, 20, 22, 23]
1357 04/05/2016 4 360584.96 1296.51 20.0 8.0 4.0 9 6
[3, 7, 8, 9, 10, 11, 12, 13, 15, 17, 18, 20, 21, 22, 23]
1356 02/05/2016 2 2307596.13 1638.07 20.0 8.0 4.0 7 8
[3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20]
1355 29/04/2016 0 0.0 2299.49 20.0 8.0 4.0 6 9
[1, 2, 3, 4, 6, 7, 8, 11, 15, 16, 18, 20, 21, 22, 24]
1354 27/04/2016 1 1351856.57 1493.02 20.0 8.0 4.0 9 6
[1, 2, 3, 5, 7, 9, 11, 12, 13, 14, 15, 18, 20, 21, 24]
1353 25/04/2016 1 1589007.19 1721.57 20.0 8.0 4.0 8 7
[3, 5, 7, 8, 9, 10, 11, 12, 14, 18, 21, 22, 23, 24, 25]
1352 22/04/2016 2 696639.39 1514.85 20.0 8.0 4.0 7 8
[6, 8, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21, 23, 24, 25]
1351 20/04/2016 3 480671.98 1152.46 20.0 8.0 4.0 8 7
[2, 4, 6, 7, 10, 11, 13, 17, 19, 20, 21, 22, 23, 24, 25]
1350 18/04/2016 1 1681533.4

1149 24/12/2014 2 719185.53 1473.77 15.0 6.0 3.0 8 7
[1, 2, 3, 6, 7, 8, 10, 12, 15, 17, 18, 19, 21, 22, 23]
1148 22/12/2014 2 1079807.5 2063.65 15.0 6.0 3.0 7 8
[1, 3, 4, 5, 6, 9, 10, 13, 14, 16, 17, 18, 19, 20, 24]
1147 19/12/2014 1 2369438.83 2422.12 15.0 6.0 3.0 7 8
[1, 6, 7, 8, 9, 11, 12, 14, 15, 16, 18, 19, 20, 22, 25]
1146 17/12/2014 4 360410.22 973.4 15.0 6.0 3.0 10 5
[2, 3, 4, 5, 7, 11, 13, 15, 17, 18, 19, 21, 22, 24, 25]
1145 15/12/2014 6 271763.87 960.77 15.0 6.0 3.0 8 7
[3, 4, 5, 6, 8, 9, 11, 13, 14, 15, 16, 17, 18, 23, 24]
1144 12/12/2014 6 241841.41 516.03 15.0 6.0 3.0 7 8
[2, 3, 4, 6, 7, 8, 10, 13, 14, 17, 18, 19, 23, 24, 25]
1143 10/12/2014 4 460903.92 1419.23 15.0 6.0 3.0 7 8
[1, 3, 4, 6, 10, 11, 12, 13, 14, 16, 17, 18, 23, 24, 25]
1142 08/12/2014 2 747916.43 786.49 15.0 6.0 3.0 6 9
[2, 4, 7, 8, 9, 10, 12, 13, 15, 16, 19, 20, 22, 23, 24]
1141 05/12/2014 4 1190925.71 1462.3 15.0 6.0 3.0 8 7
[5, 6, 9, 10, 11, 12, 13, 15, 16, 17, 18, 22, 23, 24, 25]
1140 03/12/2014 0 0.0 1

950 28/08/2013 5 265527.67 748.17 12.5 5.0 2.5 8 7
[2, 4, 5, 7, 8, 9, 12, 13, 14, 17, 18, 19, 20, 21, 25]
949 26/08/2013 3 1360692.32 1612.86 12.5 5.0 2.5 9 6
[1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 23]
948 23/08/2013 0 0.0 1344.56 12.5 5.0 2.5 8 7
[1, 2, 5, 6, 8, 9, 11, 13, 16, 17, 18, 19, 21, 22, 24]
947 21/08/2013 8 170295.8 445.23 12.5 5.0 2.5 7 8
[1, 2, 3, 5, 7, 9, 10, 11, 12, 15, 16, 18, 20, 22, 24]
946 19/08/2013 3 660967.05 1854.48 12.5 5.0 2.5 7 8
[1, 7, 8, 9, 10, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25]
945 16/08/2013 2 612444.66 963.17 12.5 5.0 2.5 9 6
[2, 3, 4, 5, 6, 9, 11, 13, 18, 19, 20, 21, 23, 24, 25]
944 14/08/2013 3 583268.77 1529.55 12.5 5.0 2.5 8 7
[1, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 21, 24, 25]
943 12/08/2013 5 344624.4 1249.86 12.5 5.0 2.5 8 7
[3, 4, 5, 6, 9, 10, 12, 13, 14, 15, 17, 18, 21, 22, 23]
942 09/08/2013 7 231612.43 1006.57 12.5 5.0 2.5 8 7
[1, 3, 4, 5, 6, 7, 11, 12, 13, 18, 19, 20, 21, 22, 24]
941 07/08/2013 2 672945.71 966.66 12.5 5

735 04/04/2012 2 767170.7 1539.8 12.5 5.0 2.5 7 8
[5, 6, 8, 10, 11, 12, 13, 14, 17, 19, 20, 21, 22, 24, 25]
734 02/04/2012 7 210189.59 984.37 12.5 5.0 2.5 8 7
[2, 3, 6, 8, 9, 10, 13, 15, 17, 18, 19, 20, 21, 23, 24]
733 30/03/2012 4 369602.29 1158.37 12.5 5.0 2.5 5 10
[1, 4, 5, 6, 7, 8, 11, 12, 14, 16, 18, 19, 20, 22, 24]
732 28/03/2012 2 732554.09 1626.27 12.5 5.0 2.5 8 7
[1, 4, 8, 9, 10, 11, 13, 14, 17, 20, 21, 22, 23, 24, 25]
731 26/03/2012 8 179014.93 676.88 12.5 5.0 2.5 7 8
[3, 4, 6, 7, 8, 11, 12, 14, 15, 17, 18, 19, 21, 22, 24]
730 23/03/2012 2 1578295.6 1068.36 12.5 5.0 2.5 6 9
[1, 3, 4, 8, 11, 12, 14, 16, 18, 19, 20, 21, 22, 24, 25]
729 21/03/2012 0 0.0 1431.36 12.5 5.0 2.5 8 7
[1, 2, 3, 4, 6, 7, 9, 10, 12, 14, 15, 17, 22, 23, 25]
728 19/03/2012 2 779365.57 1582.34 12.5 5.0 2.5 8 7
[3, 4, 6, 7, 8, 10, 11, 13, 16, 19, 20, 21, 22, 23, 25]
727 16/03/2012 2 891124.77 1641.87 12.5 5.0 2.5 7 8
[3, 4, 6, 7, 8, 9, 11, 16, 18, 20, 21, 22, 23, 24, 25]
726 14/03/2012 2 692133.56 987.77 12.

539 14/06/2010 2 701454.48 1493.78 12.5 5.0 2.5 8 7
[1, 2, 4, 5, 6, 8, 9, 12, 13, 14, 15, 16, 19, 21, 23]
538 10/06/2010 2 864843.03 2406.8 12.5 5.0 2.5 7 8
[2, 5, 6, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 24, 25]
537 07/06/2010 3 505475.29 1305.01 12.5 5.0 2.5 6 9
[1, 2, 4, 6, 9, 10, 11, 14, 15, 17, 18, 20, 22, 23, 24]
536 04/06/2010 3 358144.31 1211.77 12.5 5.0 2.5 10 5
[3, 4, 5, 7, 9, 11, 15, 16, 17, 19, 20, 22, 23, 24, 25]
535 31/05/2010 8 105001.52 343.31 12.5 5.0 2.5 11 4
[1, 4, 5, 6, 9, 11, 13, 15, 16, 17, 19, 21, 23, 24, 25]
534 27/05/2010 8 141434.12 1064.11 12.5 5.0 2.5 9 6
[1, 2, 3, 5, 7, 8, 11, 12, 13, 15, 18, 19, 21, 22, 24]
533 24/05/2010 2 825816.73 1257.27 12.5 5.0 2.5 7 8
[1, 2, 3, 8, 9, 10, 11, 13, 14, 16, 17, 18, 20, 24, 25]
532 20/05/2010 4 260411.23 745.02 12.5 5.0 2.5 8 7
[1, 2, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 19, 22, 24]
531 17/05/2010 1 1750987.0 2102.03 12.5 5.0 2.5 9 6
[1, 2, 3, 7, 10, 12, 13, 14, 15, 16, 17, 21, 22, 23, 25]
530 13/05/2010 2 921212.78 2378

314 14/04/2008 1 1364366.67 1249.42 10.0 4.0 2.0 8 7
[1, 2, 5, 6, 8, 9, 11, 12, 13, 14, 15, 19, 20, 21, 22]
313 10/04/2008 3 381993.94 1077.05 10.0 4.0 2.0 8 7
[2, 3, 6, 7, 9, 10, 12, 14, 15, 16, 17, 18, 19, 23, 25]
312 07/04/2008 1 1458512.89 1619.37 10.0 4.0 2.0 7 8
[1, 2, 6, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 22, 25]
311 03/04/2008 9 133572.09 791.41 10.0 4.0 2.0 7 8
[2, 4, 5, 7, 10, 12, 14, 15, 16, 17, 19, 20, 23, 24, 25]
310 31/03/2008 1 1396694.91 1408.43 10.0 4.0 2.0 9 6
[2, 3, 5, 9, 10, 11, 14, 15, 16, 17, 18, 19, 21, 23, 24]
309 27/03/2008 2 694576.96 1473.64 10.0 4.0 2.0 6 9
[1, 2, 3, 4, 8, 9, 10, 13, 14, 16, 17, 20, 21, 22, 24]
308 24/03/2008 2 506005.32 1060.44 10.0 4.0 2.0 8 7
[1, 2, 4, 5, 12, 13, 15, 16, 18, 19, 21, 22, 23, 24, 25]
307 20/03/2008 22 120368.22 383.76 10.0 4.0 2.0 8 7
[1, 3, 4, 6, 9, 10, 12, 14, 15, 17, 19, 20, 21, 23, 24]
306 17/03/2008 0 0.0 1852.02 10.0 4.0 2.0 7 8
[1, 2, 4, 5, 8, 11, 12, 13, 14, 15, 16, 18, 19, 20, 25]
305 13/03/2008 3 1088299.38 110

136 02/05/2006 2 671843.75 1750.35 10.0 4.0 2.0 9 6
[1, 2, 3, 5, 6, 9, 12, 13, 14, 15, 17, 21, 22, 23, 24]
135 24/04/2006 7 234154.46 906.41 10.0 4.0 2.0 8 7
[1, 2, 5, 7, 8, 10, 13, 14, 15, 18, 19, 20, 21, 22, 25]
134 17/04/2006 4 325378.91 413.15 10.0 4.0 2.0 9 6
[1, 3, 5, 7, 8, 10, 11, 12, 13, 14, 16, 17, 19, 22, 25]
133 10/04/2006 5 217132.17 517.9 10.0 4.0 2.0 10 5
[1, 3, 4, 5, 8, 9, 11, 13, 15, 18, 19, 20, 21, 22, 23]
132 03/04/2006 10 135976.17 733.01 10.0 4.0 2.0 10 5
[1, 3, 4, 5, 7, 9, 10, 11, 14, 15, 16, 17, 22, 23, 25]
131 27/03/2006 1 1341544.37 871.13 10.0 4.0 2.0 8 7
[2, 4, 5, 6, 7, 8, 11, 12, 13, 15, 19, 20, 22, 23, 25]
130 20/03/2006 3 417614.17 561.87 10.0 4.0 2.0 9 6
[2, 3, 4, 5, 6, 9, 10, 12, 13, 15, 17, 19, 23, 24, 25]
129 13/03/2006 3 448986.01 634.05 10.0 4.0 2.0 10 5
[2, 5, 6, 7, 9, 11, 13, 15, 18, 19, 20, 21, 22, 23, 25]
128 06/03/2006 2 830452.68 1686.77 10.0 4.0 2.0 9 6
[2, 3, 4, 6, 11, 13, 14, 15, 16, 17, 19, 20, 21, 23, 25]
127 01/03/2006 5 519339.5 574.65 10

In [10]:
concursos_v1 = filtrar_ultimos_concursos(concursos, 100)

In [71]:
concursos_v2 = filtrar_concurso_por_numeros(concursos, [20,21,22,23,24,25])
imprimir_concursos(concursos_v2)

Total concursos: 43 

Concurso:  89
Data Sorteio:  06/06/2005
Ganhadores 15 pontos: 2 
Impares: 9 - Pares: 6
[1, 6, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25]


Concurso:  90
Data Sorteio:  13/06/2005
Ganhadores 15 pontos: 4 
Impares: 8 - Pares: 7
[1, 2, 3, 8, 10, 11, 12, 15, 17, 20, 21, 22, 23, 24, 25]


Concurso:  161
Data Sorteio:  19/10/2006
Ganhadores 15 pontos: 0 
Impares: 9 - Pares: 6
[1, 5, 8, 11, 13, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25]


Concurso:  173
Data Sorteio:  30/11/2006
Ganhadores 15 pontos: 0 
Impares: 9 - Pares: 6
[2, 3, 4, 5, 7, 8, 11, 15, 19, 20, 21, 22, 23, 24, 25]


Concurso:  188
Data Sorteio:  22/01/2007
Ganhadores 15 pontos: 6 
Impares: 8 - Pares: 7
[2, 3, 5, 8, 10, 11, 13, 16, 19, 20, 21, 22, 23, 24, 25]


Concurso:  374
Data Sorteio:  10/11/2008
Ganhadores 15 pontos: 3 
Impares: 7 - Pares: 8
[1, 2, 3, 6, 9, 12, 13, 16, 18, 20, 21, 22, 23, 24, 25]


Concurso:  402
Data Sorteio:  16/02/2009
Ganhadores 15 pontos: 3 
Impares: 7 - Pares: 8
[1, 2, 4, 

In [216]:
par_impar_lst = []

for x in concursos:
    par_impar_lst.append([x.impar, x.par])
  
df_v2 = pd.DataFrame(par_impar_lst, columns=['impar', 'par'])
df_v2.set_index(['impar'])
# par_impar_dic = dict(par_impar_lst)
# print(par_impar_dic)

par
impar     
7        8
9        6
6        9
7        8
8        7
9        6
7        8
8        7
10       5
6        9
6        9
7        8
10       5
10       5
6        9
9        6
9        6
6        9
7        8
6        9
6        9
8        7
6        9
10       5
7        8
9        6
6        9
9        6
7        8
8        7
...    ...
7        8
7        8
5       10
9        6
7        8
8        7
7        8
9        6
7        8
6        9
9        6
9        6
9        6
7        8
8        7
7        8
9        6
6        9
8        7
8        7
8        7
8        7
7        8
9        6
6        9
9        6
6        9
7        8
5       10
7        8

[1652 rows x 1 columns]

In [229]:
df_v2_group = df_v2.groupby(['impar', 'par'])['impar'].count()

In [230]:
df_v2_group

impar  par
4      11       5
5      10      41
6      9      182
7      8      429
8      7      517
9      6      336
10     5      118
11     4       21
12     3        3
Name: impar, dtype: int64

In [231]:
df_v3 = pd.DataFrame(df_v2_group)

In [232]:
df_v3['perc'] = [(x*100)/df_v2_group.values.sum() for x in df_v2_group.values]

In [233]:
df_v3

impar       perc
impar par                  
4     11       5   0.302663
5     10      41   2.481840
6     9      182  11.016949
7     8      429  25.968523
8     7      517  31.295400
9     6      336  20.338983
10    5      118   7.142857
11    4       21   1.271186
12    3        3   0.181598

In [47]:
menor = []
maior = []
menor_maior_list = []
menor_maior_dict = {}
menor_maior_strg = []

In [48]:
for x in concursos:
    menor.append(x.numeros[0])
    maior.append(x.numeros[-1])
    menor_maior_list.append([x.numeros[0], x.numeros[-1]])
    menor_maior_strg.append(str(x.numeros[0]) + ' - ' + str(x.numeros[-1]))
#     menor_maior.update({str(x.numeros[0]) + ' - ' + str(x.numeros[-1]): 1})

In [49]:
import matplotlib.pyplot as plt

In [50]:
menor_freq = np.histogram(menor, bins=np.arange(1,11))

In [51]:
menor_perc = [(x * 100)/len(concursos) for x in menor_freq[0]]

In [52]:
menor_perc

[60.47215496368039,
 24.878934624697337,
 10.048426150121065,
 2.6634382566585955,
 1.694915254237288,
 0.24213075060532688,
 0.0,
 0.0,
 0.0]

In [19]:
np.histogram(maior, bins=np.arange(15,26))

(array([   0,    0,    0,    1,    2,    7,   20,   44,  170, 1408], dtype=int64),
 array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]))

In [36]:
menor_maior_list[:5]

[[2, 25], [1, 24], [1, 24], [1, 25], [1, 25]]

In [37]:
menor_maior_strg[:5]

['2 - 25', '1 - 24', '1 - 24', '1 - 25', '1 - 25']

In [38]:
df_menor_maior_list = pd.DataFrame(menor_maior_list, columns=['menor', 'maior'])

In [251]:
df_menor_maior_list.set_index(['menor', 'maior'])

Empty DataFrame
Columns: []
Index: [(2, 25), (1, 24), (1, 24), (1, 25), (1, 25), (1, 25), (1, 25), (1, 25), (3, 25), (2, 24), (2, 25), (1, 25), (3, 23), (1, 25), (1, 25), (2, 25), (1, 21), (2, 24), (2, 24), (3, 24), (1, 25), (1, 25), (1, 20), (1, 25), (1, 24), (5, 23), (3, 25), (1, 21), (1, 24), (1, 23), (1, 25), (1, 23), (1, 24), (1, 23), (1, 25), (1, 24), (1, 24), (1, 25), (2, 24), (1, 24), (2, 25), (1, 25), (1, 24), (3, 25), (1, 25), (1, 25), (1, 25), (2, 23), (2, 24), (1, 25), (1, 23), (1, 25), (1, 24), (2, 24), (2, 25), (1, 24), (1, 25), (3, 22), (1, 25), (1, 25), (1, 25), (1, 25), (4, 25), (1, 25), (1, 25), (1, 24), (5, 24), (1, 25), (1, 24), (1, 25), (1, 23), (1, 25), (1, 25), (1, 25), (1, 25), (1, 25), (2, 25), (1, 23), (1, 25), (1, 25), (1, 21), (1, 23), (1, 25), (1, 25), (1, 23), (2, 25), (1, 25), (1, 25), (1, 25), (1, 25), (1, 25), (1, 24), (2, 24), (2, 24), (1, 22), (3, 24), (3, 25), (1, 25), (1, 25), (1, 25), ...]

[3304 rows x 0 columns]

In [266]:
menor_maior_freq = df_menor_maior_list.groupby(['menor', 'maior']).count()

In [267]:
menor_maior_freq.values

array([], shape=(29, 0), dtype=float64)

In [147]:
menor_maior_perc = [(x*100)/len(concursos) for x in menor_maior_freq.values]

In [148]:
df = pd.DataFrame(menor_maior_freq)

In [149]:
df['percentual'] = menor_maior_perc

In [150]:
df

menor  percentual
menor maior                   
1     18         2    0.121065
      19         4    0.242131
      20         8    0.484262
      21        32    1.937046
      22        60    3.631961
      23       224   13.559322
      24       506   30.629540
      25      1162   70.338983
2     20         4    0.242131
      21         6    0.363196
      22        18    1.089588
      23        78    4.721550
      24       180   10.895884
      25       536   32.445521
3     20         2    0.121065
      22         8    0.484262
      23        28    1.694915
      24        92    5.569007
      25       202   12.227603
4     22         2    0.121065
      23         6    0.363196
      24        22    1.331719
      25        58    3.510896
5     21         2    0.121065
      23         4    0.242131
      24        12    0.726392
      25        38    2.300242
6     24         2    0.121065
      25         6    0.363196

In [138]:
df_menor_maior_str = pd.DataFrame(menor_maior_strg, columns=['col'])

In [139]:
menor_maior_group = df_menor_maior_str.groupby('col')['col'].count()

In [140]:
df_v1 = pd.DataFrame(menor_maior_group)

In [141]:
df_v1

col
col         
1 - 18     2
1 - 19     4
1 - 20     8
1 - 21    32
1 - 22    60
1 - 23   224
1 - 24   506
1 - 25  1162
2 - 20     4
2 - 21     6
2 - 22    18
2 - 23    78
2 - 24   180
2 - 25   536
3 - 20     2
3 - 22     8
3 - 23    28
3 - 24    92
3 - 25   202
4 - 22     2
4 - 23     6
4 - 24    22
4 - 25    58
5 - 21     2
5 - 23     4
5 - 24    12
5 - 25    38
6 - 24     2
6 - 25     6

In [19]:
# plt.hist(df_v1)
# plt.show()

In [20]:
# np.histogram(menor_maior)

In [ ]:
# plt.hist([menor, maior], bins=np.arange(1,26))
# plt.xlabel('Menor Numero Sorteado')
# plt.ylabel('Frequencia')
# plt.grid(True)
# plt.show()

In [53]:
total = 0

for a in apostas:
    for c in concursos:
        if len(set(a.numeros).intersection(c.numeros)) == 15:
            print(c.id)
            print(a.numeros)
            print(c.numeros)
            total += 1
            
print('total jogos ganahdores: %s' % (total))

1607
[1, 2, 3, 6, 7, 8, 9, 11, 12, 13, 14, 15, 17, 18, 24]
[1, 2, 3, 6, 7, 8, 9, 11, 12, 13, 14, 15, 17, 18, 24]
total jogos ganahdores: 1


In [142]:
numeros_aposta = []
qtd_total_apostas = 30000
qtd_apostas = 0

while (qtd_apostas < qtd_total_apostas):
    num_tmp = sorted(list(np.random.choice(np.arange(1, 26), 15, replace= False)))
    numeros_aposta.append(num_tmp)
    qtd_apostas += 1

In [143]:
print(len(numeros_aposta))

30000


In [144]:
numeros_aposta[0]

[1, 2, 3, 6, 7, 8, 10, 13, 14, 16, 18, 19, 23, 24, 25]

In [55]:
apostas_v1 = filtrar_inicio_fim(apostas, 0, [1, 2])

In [56]:
len(apostas_v1)

892

In [57]:
apostas_v2 = filtrar_inicio_fim(apostas_v1, -1, [24,25])

In [58]:
len(apostas_v2)

810

In [60]:
apostas_v3 = filtrar_impar_par(apostas_v2, [7,8,9])

In [61]:
len(apostas_v3)

617

In [65]:
apostas_v4 = filtrar_jogo_por_numeros(apostas_v3, [9], 1)

In [66]:
len(apostas_v4)

381

In [27]:
jogos_t1 = get_all_jogo()

In [28]:
len(jogos_t1)

100000

In [122]:
# imprimir_jogo(jogos_t1[:10])

In [123]:
jogos_t2 = filtrar_jogo_inicio_fim(jogos_t1, numeros=[1,3], posicao=0)
len(jogos_t2)

73390

In [124]:
jogos_t3 = filtrar_jogo_impar_par(jogos_t2, impar=[7,8,9])
len(jogos_t3)

57847

In [125]:
jogos_t4 = filtrar_jogo_por_numeros(jogos_t3, numeros=[9], match=1)
len(jogos_t4)

33756

In [126]:
jogos_t5 = filtrar_jogo_inicio_fim(jogos_t4, numeros=[24,25], posicao=-1)
len(jogos_t5)

29813

In [188]:
jogos_t6 = filtrar_jogo_por_numeros(jogos_t5, numeros=[19,20,21,22,23,24,25], match=7)
len(jogos_t6)

29794

In [191]:
jogos_t7 = filtrar_jogo_sequencia(jogos_t6, numeros=[19,20,21,22,23,24,25])
len(jogos_t7)

29456

In [192]:
jogos_t8 = filtrar_jogo_sequencia(jogos_t7, numeros=[20,21,22,23,24,25])
len(jogos_t8)

29026

In [201]:
jogos_t9 = filtrar_jogo_sequencia(jogos_t8, numeros=[1,2,3,4,5,6,7])
len(jogos_t9)

28665

In [202]:
jogos_t10 = filtrar_jogo_notin(jogos_t9, numeros=[2,23])
len(jogos_t10)

6763

In [204]:
# imprimir_jogo(jogos_t10[:20])

In [222]:
apostas = selecionar_jogos_aposta(jogos_t10, 30)

Percentual jogos com 1:  90.0
Percentual jogos com 2:  0.0
Percentual jogos com 3:  10.0
Percentual jogos impar 7:  23.333333333333332
Percentual jogos impar 8:  50.0
Percentual jogos impar 9:  26.666666666666668


In [223]:
imprimir_jogo(apostas)

Total jogos: 30 

ID:  68279
Data Cadastro:  2018-04-25
Numeros: 15 - Impares: 7 - Pares: 8
[1, 3, 4, 6, 7, 8, 9, 10, 11, 14, 16, 18, 19, 22, 25]


ID:  11163
Data Cadastro:  2018-04-25
Numeros: 15 - Impares: 8 - Pares: 7
[1, 3, 4, 5, 6, 9, 12, 14, 15, 16, 17, 18, 19, 24, 25]


ID:  68220
Data Cadastro:  2018-04-25
Numeros: 15 - Impares: 8 - Pares: 7
[1, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 18, 19, 21, 24]


ID:  47790
Data Cadastro:  2018-04-25
Numeros: 15 - Impares: 7 - Pares: 8
[3, 5, 6, 7, 8, 9, 11, 14, 15, 16, 18, 20, 22, 24, 25]


ID:  16547
Data Cadastro:  2018-04-25
Numeros: 15 - Impares: 8 - Pares: 7
[1, 5, 6, 7, 9, 10, 13, 14, 18, 19, 20, 21, 22, 24, 25]


ID:  65001
Data Cadastro:  2018-04-25
Numeros: 15 - Impares: 8 - Pares: 7
[1, 3, 4, 5, 7, 9, 12, 13, 14, 16, 20, 21, 22, 24, 25]


ID:  7108
Data Cadastro:  2018-04-25
Numeros: 15 - Impares: 9 - Pares: 6
[1, 5, 6, 8, 9, 11, 12, 13, 15, 17, 18, 20, 21, 22, 25]


ID:  99526
Data Cadastro:  2018-04-25
Numeros: 15 - Impares: 9 - 

In [224]:
for a in apostas:
    print(a.numeros)

[1, 3, 4, 6, 7, 8, 9, 10, 11, 14, 16, 18, 19, 22, 25]
[1, 3, 4, 5, 6, 9, 12, 14, 15, 16, 17, 18, 19, 24, 25]
[1, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 18, 19, 21, 24]
[3, 5, 6, 7, 8, 9, 11, 14, 15, 16, 18, 20, 22, 24, 25]
[1, 5, 6, 7, 9, 10, 13, 14, 18, 19, 20, 21, 22, 24, 25]
[1, 3, 4, 5, 7, 9, 12, 13, 14, 16, 20, 21, 22, 24, 25]
[1, 5, 6, 8, 9, 11, 12, 13, 15, 17, 18, 20, 21, 22, 25]
[1, 5, 6, 7, 8, 9, 10, 11, 15, 18, 19, 20, 21, 24, 25]
[1, 3, 5, 8, 9, 10, 12, 14, 15, 18, 19, 20, 22, 24, 25]
[1, 3, 5, 6, 8, 9, 12, 13, 14, 18, 19, 20, 21, 22, 25]
[1, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 16, 17, 22, 24]
[1, 4, 6, 7, 8, 9, 10, 13, 14, 15, 16, 19, 20, 22, 25]
[1, 3, 4, 6, 8, 9, 10, 11, 12, 14, 19, 21, 22, 24, 25]
[1, 3, 4, 5, 6, 7, 9, 10, 12, 14, 16, 17, 21, 24, 25]
[3, 4, 5, 6, 9, 10, 11, 13, 14, 15, 20, 21, 22, 24, 25]
[1, 3, 5, 9, 10, 11, 14, 15, 16, 17, 18, 19, 22, 24, 25]
[1, 4, 5, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 20, 24]
[1, 4, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 21, 25]
[1, 4

In [157]:
# gravar_jogo(apostas)

In [ ]:
# from __future__ import print_function
 
# import MySQLdb as my
 
# try:
 
#     db = my.connect(host="127.0.0.1",
#                     user="root",
#                     passwd="",
#                     db="world"
#                     )
 
#     cursor = db.cursor()
 
#     sql = "select * from city where id < 10"
#     number_of_rows = cursor.execute(sql)
 
#     print(cursor.fetchone())  # fetch the first row only
 
#     db.close()
 
# except my.DataError as e:
#     print("DataError")
#     print(e)
 
# except my.InternalError as e:
#     print("InternalError")
#     print(e)
 
# except my.IntegrityError as e:
#     print("IntegrityError")
#     print(e)
 
# except my.OperationalError as e:
#     print("OperationalError")
#     print(e)
 
# except my.NotSupportedError as e:
#     print("NotSupportedError")
#     print(e)
 
# except my.ProgrammingError as e:
#     print("ProgrammingError")
#     print(e)
 
# except :
#     print("Unknown error occurred")

In [18]:
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from reportlab.graphics.shapes import Rect
from reportlab.lib.colors import black

In [179]:
c = canvas.Canvas("formulario.pdf", pagesize=(8.3*cm, 18.8*cm))
# c.drawString(100,750, "Bem vindo ao reportlab!")
# c.setFillColor()
# c.setPageTransition(cm)

# c.rect(1.1*cm, 13.6*cm, 0.5*cm, 0.3*cm, fill=1)
# c.rect(2.4*cm, 13.6*cm, 0.5*cm, 0.3*cm, fill=1)
# c.rect(3.7*cm, 13.6*cm, 0.5*cm, 0.3*cm, fill=1)
# c.rect(5*cm, 13.6*cm, 0.5*cm, 0.3*cm, fill=1)
# c.rect(6.3*cm, 13.6*cm, 0.5*cm, 0.3*cm, fill=1)

# c.rect(1.1*cm, 13.2*cm, 0.5*cm, 0.3*cm, fill=1)
# c.rect(2.4*cm, 13.2*cm, 0.5*cm, 0.3*cm, fill=1)
# c.rect(3.7*cm, 13.2*cm, 0.5*cm, 0.3*cm, fill=1)
# c.rect(5*cm, 13.2*cm, 0.5*cm, 0.3*cm, fill=1)
# c.rect(6.3*cm, 13.2*cm, 0.5*cm, 0.3*cm, fill=1)

c.rect(1.1*cm, 8.55*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(2.4*cm, 8.55*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(3.7*cm, 8.55*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(5.0*cm, 8.55*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(6.3*cm, 8.55*cm, 0.5*cm, 0.3*cm, fill=1)

c.rect(1.1*cm, 8.1*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(2.4*cm, 8.1*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(3.7*cm, 8.1*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(5.0*cm, 8.1*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(6.3*cm, 8.1*cm, 0.5*cm, 0.3*cm, fill=1)

c.rect(1.1*cm, 7.65*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(2.4*cm, 7.65*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(3.7*cm, 7.65*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(5.0*cm, 7.65*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(6.3*cm, 7.65*cm, 0.5*cm, 0.3*cm, fill=1)

c.rect(1.1*cm, 7.2*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(2.4*cm, 7.2*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(3.7*cm, 7.2*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(5.0*cm, 7.2*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(6.3*cm, 7.2*cm, 0.5*cm, 0.3*cm, fill=1)

c.rect(1.1*cm, 6.75*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(2.4*cm, 6.75*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(3.7*cm, 6.75*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(5.0*cm, 6.75*cm, 0.5*cm, 0.3*cm, fill=1)
c.rect(6.3*cm, 6.75*cm, 0.5*cm, 0.3*cm, fill=1)

# c.rect(1,520,22,13, fill=1)
# c.rect(220,520,22,13, fill=1)
# c.rect(0,0,8.3*cm,18.8*cm, fill=0)
c.save()